# BPO

In [2]:
from koselleck import *

In [21]:
def periodize(y,ybin=5):
    y1=y//ybin*ybin
    y2=y1+ybin
    return f'{y1}-{y2}'

In [22]:
dfskip=get_skipgrams(calc_numlines=True)
dfskip['period']=dfskip.year.apply(periodize)
# dfskip=dfskip.query('1700<=year<1900')
dfskip=dfskip.query('1720<=year')  # 1725 on we have >1M words
dfskip

100%|██████████| 289/289 [00:57<00:00,  5.04it/s]


,corpus,year,path,num_lines,num_words,period
234,bpo,1720,/home/ryan/github/koselleck/data/skipgrams/yea...,4166,104150,1720-1725
270,bpo,1721,/home/ryan/github/koselleck/data/skipgrams/yea...,5656,141400,1720-1725
205,bpo,1722,/home/ryan/github/koselleck/data/skipgrams/yea...,7126,178150,1720-1725
74,bpo,1723,/home/ryan/github/koselleck/data/skipgrams/yea...,8820,220500,1720-1725
93,bpo,1724,/home/ryan/github/koselleck/data/skipgrams/yea...,6070,151750,1720-1725
...,...,...,...,...,...,...
106,bpo,1965,/home/ryan/github/koselleck/data/skipgrams/yea...,20654,516350,1965-1970
132,bpo,1966,/home/ryan/github/koselleck/data/skipgrams/yea...,21501,537525,1965-1970
116,bpo,1967,/home/ryan/github/koselleck/data/skipgrams/yea...,5316,132900,1965-1970
20,bpo,1968,/home/ryan/github/koselleck/data/skipgrams/yea...,6281,157025,1965-1970


In [23]:
# dfgg=dfskip.groupby('period').sum()
# dfgg

In [24]:
dfskip.groupby('period').num_words.sum()

period
1720-1725      795950
1725-1730     1007225
1730-1735     3335550
1735-1740     3725500
1740-1745     3197875
1745-1750     3701400
1750-1755     4202700
1755-1760     4449375
1760-1765     5439850
1765-1770     5254200
1770-1775     6778200
1775-1780     8770100
1780-1785     9480575
1785-1790    10021325
1790-1795    10028000
1795-1800    10497650
1800-1805    12494800
1805-1810    15506450
1810-1815    17915175
1815-1820    19951425
1820-1825    26663350
1825-1830    24898100
1830-1835    27177150
1835-1840    26890875
1840-1845    30750100
1845-1850    32821800
1850-1855    33975225
1855-1860    39190800
1860-1865    46852500
1865-1870    52152100
1870-1875    50863350
1875-1880    50676650
1880-1885    53240875
1885-1890    54629925
1890-1895    61195050
1895-1900    61117725
1900-1905    50766725
1905-1910    38521050
1910-1915    28646800
1915-1920    18424325
1920-1925    17074525
1925-1930    17539375
1930-1935    15295675
1935-1940     9533850
1940-1945     4525300
194

In [25]:
# dfskip[dfskip.num_lines<20000]

In [26]:
dfskipruns=pd.concat([
    dfskip.assign(run=f'run_{str(i+1).zfill(2)}')
    for i in range(1)
])
dfskipruns['opath']=dfskipruns.apply(lambda row: os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run,'model.bin'),1)
dfskipruns['opath_exists']=dfskipruns.opath.apply(lambda x: os.path.exists(x))
# dfskipruns=dfskipruns.query('run!="run_01"')
dfskipruns=dfskipruns[dfskipruns.opath_exists==False]
dfskipruns.groupby('opath_exists').size()
# dfskipruns

Series([], dtype: int64)

In [13]:
def gen_and_save_model(dfskip,nskip=DEFAULT_NUM_SKIP,force=False,vector_size=100,window=10,min_count=5,epochs=10,workers=8):
    row=dfskip.iloc[0]
    odir=os.path.join(PATH_MODELS_NEW,row.corpus,row.period,row.run)
    ofnfn=os.path.join(odir,'model.bin')
    if force or not os.path.exists(ofnfn):
        ensure_dir_exists(odir)
        ss=SkipgramsSamplers(dfskip.path, nskip)
        model = Word2Vec(sentences=ss,vector_size=vector_size,window=window,min_count=min_count,epochs=epochs,workers=workers)
        model.save(ofnfn)
    return pd.DataFrame()

# model.wv.most_similar('value',topn=25)

In [14]:
# enable_gensim_logging()
NSKIP_PER_YR=20000
# gen_and_save_model(dfskipruns.iloc[:5],force=True,nskip=NSKIP_PER_YR,epochs=10)
# m=load_model('/home/ryan/github/koselleck/data/models/bpo/1700-1705/run_01/model.bin')
# m.wv.most_similar('virtue')

In [15]:
# enable_gensim_logging()
disable_gensim_logging()

In [16]:
res=pmap_groups(
    gen_and_save_model,
    dfskipruns.groupby(['period','run']),
    num_proc=4,
    kwargs=dict(force=True, nskip=NSKIP_PER_YR)
)

Mapping gen_and_save_model [x4]: 100%|██████████| 14/14 [02:22<00:00, 10.14s/it]


In [17]:
m=load_model('/home/ryan/github/koselleck/data/models/bpo/1905-1910/run_01/model.bin')
m.wv.most_similar('virtue')

[('ignorance', 0.7392162680625916),
 ('environment', 0.7112278342247009),
 ('greatness', 0.6904208064079285),
 ('intellect', 0.6891406178474426),
 ('motive', 0.6839988231658936),
 ('power', 0.677443265914917),
 ('welfare', 0.6711458563804626),
 ('patriotism', 0.6694182753562927),
 ('limitations', 0.6671348810195923),
 ('belief', 0.6654924154281616)]